In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit

## QUANTILE REGRESSION

In [2]:
data = pd.read_csv('non_nan.csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [3]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)
    
    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [4]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [5]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

In [6]:
# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [7]:
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag1,NVO_lag2,NVS_lag1,NVS_lag2,PFE_lag1,PFE_lag2,ROG_lag1,ROG_lag2,Quater_dummy,Quater_rippel
0,2000-07-01,4.0,172.700,6.097273,-0.653855,-1.338453,-8.064479,-11.373392,-8.650271,4.005041,...,2.813690,20.863985,8.474599,5.169310,8.076012,5.638019,13.131313,-8.163265,1,0
1,2000-08-01,4.1,172.700,6.054000,0.795710,6.534124,6.578935,3.131245,-1.192113,-29.708189,...,13.757382,2.813690,-3.593737,8.474599,-9.635421,8.076012,2.142857,13.131313,0,0
2,2000-09-01,3.9,173.600,5.826087,0.249670,-5.723123,15.363525,8.458631,2.496165,11.482185,...,5.591665,13.757382,-1.944901,-3.593737,-0.540360,-9.635421,-2.447552,2.142857,1,0
3,2000-10-01,3.9,173.900,5.799000,0.187218,-0.217758,-8.764862,5.978226,-1.929509,10.169493,...,9.359631,5.591665,2.314016,-1.944901,4.352595,-0.540360,-9.318996,-2.447552,1,0
4,2000-11-01,3.9,174.200,5.738571,0.009919,-7.465269,7.863717,14.237843,8.548194,4.825204,...,-4.279274,9.359631,-0.807726,2.314016,-3.860903,4.352595,13.043478,-9.318996,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,8.619586,14.479974,1.295221,10.390734,3.090302,7.690670,9.446071,-53.665066,1,0
271,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,2.541749,8.619586,-0.110227,1.295221,-13.817335,3.090302,16.968326,9.446071,0,0
272,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,1.592445,2.541749,-7.172811,-0.110227,-7.286115,-13.817335,5.451681,16.968326,1,0
273,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,12.873250,1.592445,9.367574,-7.172811,0.566924,-7.286115,11.025813,5.451681,1,0


### Defining tick_loss function

In [8]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

In [9]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

# Naïve Models Specifications

## 1. Macros Only

We are only going to train the model with the lagged macroeconomic variables

In [10]:
data_macro = data.copy()
to_exclude = []

for t in tickers:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

In [11]:
to_exclude

['AZN',
 'AZN_lag1',
 'AZN_lag2',
 'BMY',
 'BMY_lag1',
 'BMY_lag2',
 'JNJ',
 'JNJ_lag1',
 'JNJ_lag2',
 'LLY',
 'LLY_lag1',
 'LLY_lag2',
 'MRK',
 'MRK_lag1',
 'MRK_lag2',
 'NVO',
 'NVO_lag1',
 'NVO_lag2',
 'NVS',
 'NVS_lag1',
 'NVS_lag2',
 'PFE',
 'PFE_lag1',
 'PFE_lag2',
 'ROG',
 'ROG_lag1',
 'ROG_lag2',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'Date']

In [12]:
data_macro_X = data.drop(columns=to_exclude)

In [13]:
data_macro_X

,unemployment_rate_lag1,unemployment_rate_lag2,unemployment_rate_lag3,CPI_lag1,CPI_lag2,CPI_lag3,treasury_yield_lag1,treasury_yield_lag2,treasury_yield_lag3,GDP_growth_lag1,...,inflation_change_lag2,inflation_change_lag3,unemp_change_lag1,unemp_change_lag2,unemp_change_lag3,treasury_yield_change_lag1,treasury_yield_change_lag2,treasury_yield_change_lag3,Quater_dummy,Quater_rippel
0,4.0,4.0,3.8,172.200,171.200,170.900,6.440455,5.990526,6.256522,0.305565,...,0.300,-0.100,0.0,0.2,-0.2,0.449928,-0.265995,-0.262978,1,0
1,4.0,4.0,4.0,172.700,172.200,171.200,6.097273,6.440455,5.990526,-0.653855,...,1.000,0.300,0.0,0.0,0.2,-0.343182,0.449928,-0.265995,0,0
2,4.1,4.0,4.0,172.700,172.700,172.200,6.054000,6.097273,6.440455,0.795710,...,0.500,1.000,0.1,0.0,0.0,-0.043273,-0.343182,0.449928,1,0
3,3.9,4.1,4.0,173.600,172.700,172.700,5.826087,6.054000,6.097273,0.249670,...,0.000,0.500,-0.2,0.1,0.0,-0.227913,-0.043273,-0.343182,1,0
4,3.9,3.9,4.1,173.900,173.600,172.700,5.799000,5.826087,6.054000,0.187218,...,0.900,0.000,0.0,-0.2,0.1,-0.027087,-0.227913,-0.043273,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,3.5,3.6,3.7,298.990,298.598,297.987,3.891000,3.983500,3.519048,-0.085161,...,0.611,1.448,-0.1,-0.1,0.2,-0.092500,0.464452,0.621222,1,0
271,3.4,3.5,3.6,300.536,298.990,298.598,3.616190,3.891000,3.983500,0.390254,...,0.392,0.611,-0.1,-0.1,-0.1,-0.274810,-0.092500,0.464452,0,0
272,3.6,3.4,3.5,301.648,300.536,298.990,3.531500,3.616190,3.891000,-0.442183,...,1.546,0.392,0.2,-0.1,-0.1,-0.084690,-0.274810,-0.092500,1,0
273,3.5,3.6,3.4,301.808,301.648,300.536,3.746842,3.531500,3.616190,-0.442183,...,1.112,1.546,-0.1,0.2,-0.1,0.215342,-0.084690,-0.274810,1,0


### 1-step-ahead VaR: 1 month

In [14]:
def rolling_1m_forecast(data, dates, ticker, alpha, columns_to_exclude):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/12

    return t_loss  

In [15]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

In [16]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-1m-loss: {loss} \n')

AZN-1m-loss: 0.604547859797446 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-1m-loss: 0.6347272095304196 

JNJ-1m-loss: 0.6175764384728927 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-1m-loss: 1.8786960638489365 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-1m-loss: 0.7852638891416645 

NVO-1m-loss: 0.822291454625585 

NVS-1m-loss: 1.0584249789278957 

PFE-1m-loss: 1.4214312297640215 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 4.980231802811116 



In [17]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

### 3-step-ahead VaR: 3 month

In [18]:
def rolling_3m_forecast(data, dates, ticker, alpha, columns_to_exclude):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/10

    return t_loss  

In [19]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [20]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-3m-loss: {loss} \n')

AZN-3m-loss: 0.5916389691501852 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-3m-loss: 0.6492955386209205 

JNJ-3m-loss: 0.6073184422020041 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-3m-loss: 2.001256535681705 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-3m-loss: 0.7805126455568807 

NVO-3m-loss: 0.9160228301882686 

NVS-3m-loss: 1.1235852856670652 

PFE-3m-loss: 1.6032114231682162 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-3m-loss: 5.729698918808312 



### 6-steap-ahead VaR: 6 months

In [21]:
def rolling_6m_forecast(data, dates, ticker, alpha, columns_to_exclude):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/7

    return t_loss  

In [22]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [23]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-6m-loss: {loss} \n')

AZN-6m-loss: 0.6107010716604047 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-6m-loss: 0.6718311030453836 

JNJ-6m-loss: 0.6560277321851667 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-6m-loss: 2.5848315967921645 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-6m-loss: 0.8282797537027216 

NVO-6m-loss: 0.8895784370850747 

NVS-6m-loss: 1.3646992593776905 

PFE-6m-loss: 1.7576163329131131 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-6m-loss: 7.557972287902161 



### 9-step-ahead VaR: 9 month

In [24]:
def rolling_9m_forecast(data, dates, ticker, alpha, columns_to_exclude):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/4

    return t_loss  

In [25]:
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [26]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-9m-loss: {loss} \n')

AZN-9m-loss: 0.49429362887607864 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-9m-loss: 0.58904073582743 

JNJ-9m-loss: 0.7571721725978307 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-9m-loss: 3.997181482796207 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-9m-loss: 0.7023878666894035 

NVO-9m-loss: 0.7256023122979017 

NVS-9m-loss: 1.8077781557215615 

PFE-9m-loss: 2.419061177759293 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-9m-loss: 0.9826010976007642 



### 12-step-ahead VaR: 12 months

In [27]:
def rolling_12m_forecast(data, dates, ticker, alpha, columns_to_exclude):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/1

    return t_loss  

In [28]:
dates12m = ['2022-05-01']

In [29]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-12m-loss: {loss} \n')

AZN-12m-loss: 1.9771745155043146 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-12m-loss: 2.35616294330972 

JNJ-12m-loss: 3.0286886903913226 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-12m-loss: 15.988725931184828 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-12m-loss: 2.809551466757614 

NVO-12m-loss: 2.902409249191607 

NVS-12m-loss: 7.231112622886246 

PFE-12m-loss: 9.676244711037173 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-12m-loss: 3.930404390403057 



# 2. Ticker data

In [30]:
data_stocks = data.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')

In [31]:
to_exclude2

['unemployment_rate',
 'unemployment_rate_lag1',
 'unemployment_rate_lag2',
 'unemployment_rate_lag3',
 'CPI',
 'CPI_lag1',
 'CPI_lag2',
 'CPI_lag3',
 'treasury_yield',
 'treasury_yield_lag1',
 'treasury_yield_lag2',
 'treasury_yield_lag3',
 'GDP_growth',
 'GDP_growth_lag1',
 'GDP_growth_lag2',
 'GDP_growth_lag3',
 'SP500_return',
 'SP500_return_lag1',
 'SP500_return_lag2',
 'SP500_return_lag3',
 'inflation_change',
 'inflation_change_lag1',
 'inflation_change_lag2',
 'inflation_change_lag3',
 'unemp_change',
 'unemp_change_lag1',
 'unemp_change_lag2',
 'unemp_change_lag3',
 'treasury_yield_change',
 'treasury_yield_change_lag1',
 'treasury_yield_change_lag2',
 'treasury_yield_change_lag3',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG',
 'Date']

### 1-step-ahead VaR

In [32]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

### 1-step-ahead VaR: 1 month

In [33]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-1m-loss: {loss} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 1.1225649518052998 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-1m-loss: 1.8317494200049766 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 1.4404516641720473 

LLY-1m-loss: 1.73391108156547 

MRK-1m-loss: 0.8931670009261032 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-1m-loss: 1.4660887417504542 

NVS-1m-loss: 1.814312476719338 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-1m-loss: 1.3782630727748366 

ROG-1m-loss: 5.805699770777661 



### 3-step-ahead VaR: 3 months

In [34]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-3m-loss: {loss} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-3m-loss: 1.214013772004391 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-3m-loss: 2.1098116164159886 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-3m-loss: 1.6642611273723182 

LLY-3m-loss: 1.8492009759742913 

MRK-3m-loss: 0.9421248971845341 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-3m-loss: 1.504596399518301 

NVS-3m-loss: 2.0371065520768767 

PFE-3m-loss: 1.613634808360158 

ROG-3m-loss: 6.88840727021267 



### 6-step-ahead VaR: 6 months

In [35]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-6m-loss: {loss} \n')

AZN-6m-loss: 1.496580906741892 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-6m-loss: 1.5147983464386594 

JNJ-6m-loss: 1.3499264564586628 

LLY-6m-loss: 1.4857937740867915 

MRK-6m-loss: 0.9783708665533337 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-6m-loss: 1.8696950460326618 

NVS-6m-loss: 1.2348587747139532 

PFE-6m-loss: 1.1344532376345173 

ROG-6m-loss: 9.209879374741481 



### 9-step-ahead VaR: 9 months

In [36]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-9m-loss: {loss} \n')

AZN-9m-loss: 2.093000421150526 

BMY-9m-loss: 2.075764541176609 

JNJ-9m-loss: 1.979369976505056 

LLY-9m-loss: 2.1390758923892403 

MRK-9m-loss: 1.0001734223743326 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-9m-loss: 2.497249229276347 

NVS-9m-loss: 1.5927917468805082 

PFE-9m-loss: 1.5387291599674715 

ROG-9m-loss: 2.507091304153901 



### 12-step-ahead VaR: 12 months

In [37]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-12m-loss: {loss} \n')

AZN-12m-loss: 0.42698593060488266 

BMY-12m-loss: 0.7878275362073656 

JNJ-12m-loss: 0.26906442981152384 

LLY-12m-loss: 0.5367435768730582 

MRK-12m-loss: 0.31858924307554104 

NVO-12m-loss: 0.4911857679729013 

NVS-12m-loss: 1.199923909376334 

PFE-12m-loss: 0.12424211221831577 

ROG-12m-loss: 0.8642262897925357 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


# 3. All Covariates

In [38]:
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

### 1-step-ahead VaR: 1 months

In [39]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude3)
    print(f'{t}-1m-loss: {loss} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 0.8967213830757417 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-1m-loss: 0.7146737131818566 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

NVS-1m-loss: 1.4996473162536874 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-1m-loss: 2.310692307079258 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 4.499513496348546 



### 3-step-ahead VaR: 3 months

In [40]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude3)
    print(f'{t}-3m-loss: {loss} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-3m-loss: 0.914923206238359 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-3m-loss: 0.7514119771238522 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

NVS-3m-loss: 1.6205957941980962 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-3m-loss: 2.6770172621566974 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-3m-loss: 4.874091226899671 



### 6-step-ahead VaR: 6 months

In [41]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude3)
    print(f'{t}-6m-loss: {loss} \n')

AZN-6m-loss: 1.0517750869462354 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-6m-loss: 0.5561498798584777 

JNJ-6m-loss: 3.592723912075155 

LLY-6m-loss: 1.7088307462435566 

MRK-6m-loss: 0.8181740317200336 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-6m-loss: 0.6874720353212502 



...

NVS-6m-loss: 1.9305251279888853 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-6m-loss: 2.4621425175611162 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-6m-loss: 6.343190089956562 



### 9-step-ahead VaR: 9 months

In [42]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude3)
    print(f'{t}-9m-loss: {loss} \n')

AZN-9m-loss: 1.2114613976021742 

BMY-9m-loss: 0.30927914116211386 

JNJ-9m-loss: 5.8925278192795165 

LLY-9m-loss: 2.610294817990406 

MRK-9m-loss: 0.5331058524548647 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-9m-loss: 0.5484833581039475 

NVS-9m-loss: 2.7306882831564803 

PFE-9m-loss: 3.75248270942117 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-9m-loss: 2.2044555280870406 



### 12-step-ahead VaR: 12 months

In [43]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude3)
    print(f'{t}-12m-loss: {loss} \n')

AZN-12m-loss: 0.46102723730800454 

BMY-12m-loss: 0.5744525788949154 

JNJ-12m-loss: 3.051485905869477 

LLY-12m-loss: 0.3036732176000039 

MRK-12m-loss: 0.17210867204662755 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-12m-loss: 0.49495454238229997 

NVS-12m-loss: 4.322700597305763 

PFE-12m-loss: 0.10402411527543137 

ROG-12m-loss: 4.376838235850442 



``READAPT FROM HERE TO THE END OF THE NOTEBOOK``

### ABBV case

In [44]:
# ABBV case
data = pd.read_csv('non_nan.csv', )
data2 = pd.read_csv('monthly_data.csv')
data_ABBV = data2[['Date', 'ABBV']]
data_ABBV = data_ABBV.dropna()
data_ABBV

,Date,ABBV
158,2013-03-01,10.455062
159,2013-04-01,12.922975
160,2013-05-01,-6.438455
161,2013-06-01,-3.162299
162,2013-07-01,10.014486
...,...,...
276,2023-01-01,-8.576205
277,2023-02-01,5.152609
278,2023-03-01,3.554254
279,2023-04-01,-5.176628


In [45]:
merged_df = data_ABBV.merge(data, on='Date', how='inner')
merged_df

,Date,ABBV,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2013-03-01,10.455062,7.5,232.282,1.984211,0.397212,3.337507,14.582205,11.414669,7.978126,4.851792,3.440210,-7.714279,9.067089,5.443922,-0.188640,-0.655,-0.2,0.069449
1,2013-04-01,12.922975,7.6,231.797,1.957500,0.085246,2.374537,3.881563,-3.568848,4.538222,-2.482836,7.359867,11.497419,3.537347,0.727651,-10.457790,-0.485,0.1,-0.026711
2,2013-05-01,-6.438455,7.5,231.893,1.759091,-0.190605,2.361004,-1.290451,16.816781,-1.231989,-4.008663,-0.638313,-8.792401,-2.711502,-6.329562,8.513136,0.096,-0.1,-0.198409
3,2013-06-01,-3.162299,7.5,232.445,1.928182,0.620501,-1.853781,-7.707318,-2.868894,2.762464,-6.788137,-0.535372,-3.805087,-1.463219,3.722003,2.269287,0.552,0.0,0.169091
4,2013-07-01,10.014486,7.3,232.900,2.300000,0.210359,5.724173,7.230424,-3.244595,8.898141,8.122938,4.655361,9.001745,1.272809,4.355584,17.540151,0.455,-0.2,0.371818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-01-01,-8.576205,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
119,2023-02-01,5.152609,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
120,2023-03-01,3.554254,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
121,2023-04-01,-5.176628,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [46]:
merged_df.columns

Index(['Date', 'ABBV', 'unemployment_rate', 'CPI', 'treasury_yield',
       'GDP_growth', 'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO',
       'NVS', 'PFE', 'ROG', 'inflation_change', 'unemp_change',
       'treasury_yield_change'],
      dtype='object')

In [47]:
# Creating lagged variables
tickers_plus_ABBV = ['ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

merged_df, lagged_covs = create_lags_cov(merged_df, covs, period=period_covs)
merged_df, lagged_tickers = create_lags_returns(merged_df, tickers_plus_ABBV, period= period_returns)

In [48]:
# Creating dummy variables
merged_df['Quater_dummy'] = 0
merged_df['Quater_rippel'] = 0
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
for index, row in merged_df.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        merged_df.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        merged_df.loc[index, 'Quater_dummy'] = 1

In [49]:
merged_df.columns

Index(['Date', 'ABBV', 'unemployment_rate', 'CPI', 'treasury_yield',
       'GDP_growth', 'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO',
       'NVS', 'PFE', 'ROG', 'inflation_change', 'unemp_change',
       'treasury_yield_change', 'unemployment_rate_lag1',
       'unemployment_rate_lag2', 'unemployment_rate_lag3', 'CPI_lag1',
       'CPI_lag2', 'CPI_lag3', 'treasury_yield_lag1', 'treasury_yield_lag2',
       'treasury_yield_lag3', 'GDP_growth_lag1', 'GDP_growth_lag2',
       'GDP_growth_lag3', 'SP500_return_lag1', 'SP500_return_lag2',
       'SP500_return_lag3', 'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'ABBV_lag1',
       'ABBV_lag2', 'AZN_lag1', 'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1',
       'JNJ_lag2', 'LLY_lag1', 'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1

## 1. Macros Only

In [50]:
to_exclude = []

for t in tickers_plus_ABBV:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

In [51]:
to_exclude

['ABBV',
 'ABBV_lag1',
 'ABBV_lag2',
 'AZN',
 'AZN_lag1',
 'AZN_lag2',
 'BMY',
 'BMY_lag1',
 'BMY_lag2',
 'JNJ',
 'JNJ_lag1',
 'JNJ_lag2',
 'LLY',
 'LLY_lag1',
 'LLY_lag2',
 'MRK',
 'MRK_lag1',
 'MRK_lag2',
 'NVO',
 'NVO_lag1',
 'NVO_lag2',
 'NVS',
 'NVS_lag1',
 'NVS_lag2',
 'PFE',
 'PFE_lag1',
 'PFE_lag2',
 'ROG',
 'ROG_lag1',
 'ROG_lag2',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'Date']

In [58]:
# 1 month
loss_1m = rolling_1m_forecast(merged_df, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude)
print(f'{t}-1m-loss-ABBV: {loss_1m} \n')
# 3 months
loss_3m = rolling_3m_forecast(merged_df, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude)
print(f'{t}-3m-loss-ABBV: {loss_3m} \n')
# 6 months
loss_6m = rolling_6m_forecast(merged_df, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude)
print(f'{t}-6m-loss-ABBV: {loss_6m} \n')
# 9 months
loss_9m = rolling_9m_forecast(merged_df, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude)
print(f'{t}-9m-loss-ABBV: {loss_9m} \n')
# 12 months
loss_12m = rolling_12m_forecast(merged_df, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude)
print(f'{t}-12m-loss-ABBV: {loss_12m} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss-ABBV: 4.372088516179196 

ROG-3m-loss-ABBV: 5.077277267911996 

ROG-6m-loss-ABBV: 6.578183454068367 

ROG-9m-loss-ABBV: 3.266394882941529 

ROG-12m-loss-ABBV: 2.7337800287346385 



## 2. Ticker data

In [59]:
merged_df_stocks = merged_df.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')

In [65]:
# 1 month
loss_1m = rolling_1m_forecast(merged_df_stocks, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude2)
print(f'{t}-1m-loss-ABBV: {loss_1m} \n')

# 3 months
loss_3m = rolling_1m_forecast(merged_df_stocks, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude2)
print(f'{t}-3m-loss-ABBV: {loss_3m} \n')

# 6 months
loss_6m = rolling_6m_forecast(merged_df_stocks, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude2)
print(f'{t}-6m-loss-ABBV: {loss_6m} \n')

# 9 months
loss_9m = rolling_9m_forecast(merged_df_stocks, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude2)
print(f'{t}-9m-loss-ABBV: {loss_9m} \n')

# 12 months
loss_9m = rolling_12m_forecast(merged_df_stocks, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude2)
print(f'{t}-12m-loss-ABBV: {loss_12m} \n')

ROG-1m-loss-ABBV: 6.417290630882615 

ROG-3m-loss-ABBV: 6.377780609659756 

ROG-6m-loss-ABBV: 10.690886858689497 

ROG-9m-loss-ABBV: 5.268007431563019 

ROG-12m-loss-ABBV: 2.7337800287346385 



## 3. All Covariates

In [67]:
merged_df_covs = merged_df.copy()
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

In [68]:
# 1 month
loss_1m = rolling_1m_forecast(merged_df_covs, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude3)
print(f'{t}-1m-loss-ABBV: {loss_1m} \n')

# 3 month
loss_3m = rolling_3m_forecast(merged_df_covs, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude3)
print(f'{t}-3m-loss-ABBV: {loss_3m} \n')

# 6 month
loss_6m = rolling_6m_forecast(merged_df_covs, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude3)
print(f'{t}-6m-loss-ABBV: {loss_6m} \n')

# 9 month
loss_9m = rolling_9m_forecast(merged_df_covs, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude3)
print(f'{t}-9m-loss-ABBV: {loss_9m} \n')

# 12 month
loss_12m = rolling_12m_forecast(merged_df_covs, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude3)
print(f'{t}-12m-loss-ABBV: {loss_12m} \n')

ROG-1m-loss-ABBV: 7.280420646835054 

ROG-3m-loss-ABBV: 8.554346630674502 

ROG-6m-loss-ABBV: 11.599433799942863 

ROG-9m-loss-ABBV: 9.888351333008542 

ROG-12m-loss-ABBV: 7.768957805456391 



## QUANTILE REGRESSION VS BASELINE

### 1 month

In [93]:
data_1m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [0.416, 0.587, 0.725, 0.620, 0.518, 0.609, 0.735, 0.588, 0.899, 3.772],
    'QR Macro Only': [1.058, 4.372, 0.605, 0.635, 0.618, 1.879, 0.785, 0.822, 1.421, 4.980],
    'QR Stock Only': [1.814, 6.417, 1.123, 1.832, 1.440, 1.734, 0.893, 1.466, 1.378, 5.806],
    'QR All': [ 1.500, 6.417, 0.897, 0.715, 2.314, 1.367, 0.655, 0.744, 2.311, 4.500]
}

df_1M= pd.DataFrame(data_1m)
df_1M

,Ticker,Historical Quantile,QR Macro Only,QR Stock Only,QR All
0,NVS,0.416,1.058,1.814,1.500
1,ABBV,0.587,4.372,6.417,6.417
2,AZN,0.725,0.605,1.123,0.897
3,BMY,0.620,0.635,1.832,0.715
4,JNJ,0.518,0.618,1.440,2.314
5,LLY,0.609,1.879,1.734,1.367
6,MRK,0.735,0.785,0.893,0.655
7,NVO,0.588,0.822,1.466,0.744
8,PFE,0.899,1.421,1.378,2.311
9,ROG,3.772,4.980,5.806,4.500


### 3 months

In [94]:
data_3m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [1.299, 1.731, 2.164, 1.873, 1.443, 1.807, 2.215, 1.730, 2.394, 12.951],
    'QR Macro Only': [1.124, 5.077, 0.592, 0.649, 0.607, 2.001, 0.781, 0.916, 1.603, 5.730],
    'QR Stock Only': [2.037,  6.378, 1.214, 2.200, 1.664, 1.849, 0.942, 1.505, 1.614, 6.888],
    'QR All': [1.621, 8.554, 0.914, 0.751, 2.636, 1.431, 0.700, 0.766, 2.677, 4.874]
}

df_3M = pd.DataFrame(data_3m)

df_3M

,Ticker,Historical Quantile,QR Macro Only,QR Stock Only,QR All
0,NVS,1.299,1.124,2.037,1.621
1,ABBV,1.731,5.077,6.378,8.554
2,AZN,2.164,0.592,1.214,0.914
3,BMY,1.873,0.649,2.200,0.751
4,JNJ,1.443,0.607,1.664,2.636
5,LLY,1.807,2.001,1.849,1.431
6,MRK,2.215,0.781,0.942,0.700
7,NVO,1.730,0.916,1.505,0.766
8,PFE,2.394,1.603,1.614,2.677
9,ROG,12.951,5.730,6.888,4.874


### 6 months

In [95]:
data_6m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [2.731, 3.991, 5.725, 4.406, 3.895, 4.285, 5.170, 3.677, 4.776, 26.300],
    'QR Macro Only': [1.365, 6.578, 0.611, 0.672, 0.656, 2.585, 0.828, 0.890, 1.758, 7.558],
    'QR Stock Only': [1.235, 10.691, 1.497, 1.515, 1.350, 1.486, 0.978, 1.800, 1.134, 9.210],
    'QR All': [1.931, 11.599, 1.052, 0.556, 3.593, 1.708, 0.818, 0.687, 2.462, 6.343]
}

df_6M = pd.DataFrame(data_6m)

df_6M

,Ticker,Historical Quantile,QR Macro Only,QR Stock Only,QR All
0,NVS,2.731,1.365,1.235,1.931
1,ABBV,3.991,6.578,10.691,11.599
2,AZN,5.725,0.611,1.497,1.052
3,BMY,4.406,0.672,1.515,0.556
4,JNJ,3.895,0.656,1.350,3.593
5,LLY,4.285,2.585,1.486,1.708
6,MRK,5.170,0.828,0.978,0.818
7,NVO,3.677,0.890,1.800,0.687
8,PFE,4.776,1.758,1.134,2.462
9,ROG,26.300,7.558,9.210,6.343


### 9 months

In [96]:
data_9m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [2.731, 3.991, 5.725, 4.406, 3.895, 4.285, 5.170, 3.677, 4.776, 26.300],
    'QR Macro Only': [1.808, 3.266, 0.494, 0.589, 0.757, 3.997, 0.702, 0.726, 2.419, 0.983],
    'QR Stock Only': [1.593, 5.268, 2.093, 2.075, 1.979, 2.139, 1.000, 2.497, 1.539, 2.507],
    'QR All': [2.731, 9.888, 1.211, 0.309, 5.892, 2.610, 0.533, 0.548, 3.752, 2.204]
}

df_9M = pd.DataFrame(data_9m)
df_9M

,Ticker,Historical Quantile,QR Macro Only,QR Stock Only,QR All
0,NVS,2.731,1.808,1.593,2.731
1,ABBV,3.991,3.266,5.268,9.888
2,AZN,5.725,0.494,2.093,1.211
3,BMY,4.406,0.589,2.075,0.309
4,JNJ,3.895,0.757,1.979,5.892
5,LLY,4.285,3.997,2.139,2.610
6,MRK,5.170,0.702,1.000,0.533
7,NVO,3.677,0.726,2.497,0.548
8,PFE,4.776,2.419,1.539,3.752
9,ROG,26.300,0.983,2.507,2.204


### 12 months

In [97]:
data_12m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [5.275, 7.109, 8.276, 7.670, 5.860, 7.081, 8.847, 7.067, 10.229, 45.244],
    'QR Macro Only': [7.231, 2.734, 1.977, 2.356, 3.029, 15.989, 2.809, 2.902, 9.676, 3.930],
    'QR Stock Only': [1.200, 2.733, 0.427, 0.788, 0.269, 0.537, 0.318, 0.491, 0.124, 0.864],
    'QR All': [4.323, 7.768, 0.461, 0.574, 3.051, 0.304, 0.172,0.495,0.104, 4.376]
}

df_12M = pd.DataFrame(data_12m)
df_12M

,Ticker,Historical Quantile,QR Macro Only,QR Stock Only,QR All
0,NVS,5.275,7.231,1.200,4.323
1,ABBV,7.109,2.734,2.733,7.768
2,AZN,8.276,1.977,0.427,0.461
3,BMY,7.670,2.356,0.788,0.574
4,JNJ,5.860,3.029,0.269,3.051
5,LLY,7.081,15.989,0.537,0.304
6,MRK,8.847,2.809,0.318,0.172
7,NVO,7.067,2.902,0.491,0.495
8,PFE,10.229,9.676,0.124,0.104
9,ROG,45.244,3.930,0.864,4.376
